# Step 4: Analyze results

In [ ]:
import pycompwa.ui as pwa

In [ ]:
pwa.Logging('error')  # at this stage, we are not interested in the back-end
particle_list = pwa.read_particles('model.xml')
kinematics = pwa.create_helicity_kinematics('model.xml', particle_list)
kinematics.create_all_subsystems()
data_sample = pwa.read_root_data(input_file='generated_data.root')
phsp_sample = pwa.read_root_data(input_file='generated_phsp.root')
intensity_builder = pwa.IntensityBuilderXML(
    'model.xml', particle_list, kinematics, phsp_sample)
fit_result = pwa.load('fit_result.xml')
intensity = intensity_builder.create_intensity()
data_set = kinematics.convert(data_sample)
phsp_set = kinematics.convert(phsp_sample)
estimator, initial_parameters = \
    pwa.create_unbinned_log_likelihood_function_tree_estimator(
        intensity, data_set)
intensity.updateParametersFrom(fit_result.final_parameters)

## 4.1 Convert to pandas

In [ ]:
from pycompwa.data import convert

In [ ]:
frame_data = convert.data_set_to_pandas(data_set)
frame_phsp = convert.data_set_to_pandas(phsp_set)
frame_data

In [ ]:
intensity_set = intensity.evaluate(phsp_set.data)
frame_phsp['weights'] = intensity_set

## 4.2 Visualize

### Kinematic variables

In [ ]:
from pycompwa.data import naming

naming.replace_ids('2,3', 'model.xml')

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def plot_1d_comparison(name, bins=100, **kwargs):
    """Helper function for comparing the 1D distributions of fit and data."""
    frame_data[name].hist(bins=bins, density=True,
                          alpha=.5, label='data', **kwargs)
    frame_phsp[name].hist(bins=bins, weights=frame_phsp['weights'],
                          density=True, histtype='step', color='red',
                          label='fit', **kwargs)
    plt.ylabel('normalized intensity')
    title = naming.replace_ids(name, kinematics)
    plt.xlabel(title)

In [ ]:
plot_1d_comparison('theta_2_4_vs_3')
plt.legend();

In [ ]:
from numpy import sqrt

frame_data['M(3,4)'] = sqrt(frame_data['mSq_(3,4)'])
frame_phsp['M(3,4)'] = sqrt(frame_phsp['mSq_(3,4)'])

plot_1d_comparison('M(3,4)')
plt.legend();

### Dalitz plots

Dalitz plots are 2-dimensional histograms of the square values of the
invariant masses. We are therefore interested in the following variables:

In [ ]:
variable_names = data_set.data.keys()
[var for var in variable_names if var.startswith('mSq_')]

In [ ]:
def dalitz_plot(frame, mass_x, mass_y, bins=50, **kwargs):
    """Helper function to create a Dalitz plot with useful axis titles."""
    plt.hist2d(
        frame[mass_x],
        frame[mass_y],
        bins=bins,
        **kwargs)
    plt.xlabel(naming.replace_ids(mass_x, kinematics))
    plt.ylabel(naming.replace_ids(mass_y, kinematics))

In [ ]:
from matplotlib.colors import LogNorm  # logarithmic z-axis

fig, axs = plt.subplots(1, 2, figsize=(9, 4))

plt.sca(axs[0])
axs[0].set_title('data')
dalitz_plot(frame_data, 'mSq_(3,4)', 'mSq_(2,4)', norm=LogNorm())

plt.sca(axs[1])
axs[1].set_title('fit')
dalitz_plot(frame_phsp, 'mSq_(3,4)', 'mSq_(2,4)', norm=LogNorm(),
            weights=frame_phsp['weights'])

## 4.3 Calculate fit fractions

In [ ]:
intensity_builder.get_all_component_names()

As you can see, these components are amplitudes and intensities that are
added coherently or incoherently.

In [ ]:
components = intensity_builder.create_intensity_components([
    ['coherent_J/psi_-1_to_gamma_-1+pi0_0+pi0_0'],
    ['J/psi_-1_to_f2(1270)_-1+gamma_-1;f2(1270)_-1_to_pi0_0+pi0_0;'],
    ['J/psi_-1_to_f2(1270)_-2+gamma_-1;f2(1270)_-2_to_pi0_0+pi0_0;'],
    ['J/psi_-1_to_f2(1270)_0+gamma_-1;f2(1270)_0_to_pi0_0+pi0_0;'],
    [
        'J/psi_-1_to_f2(1270)_-1+gamma_-1;f2(1270)_-1_to_pi0_0+pi0_0;',
        'J/psi_-1_to_f2(1270)_-2+gamma_-1;f2(1270)_-2_to_pi0_0+pi0_0;',
        'J/psi_-1_to_f2(1270)_0+gamma_-1;f2(1270)_0_to_pi0_0+pi0_0;',
    ],
])

In [ ]:
fit_fractions = pwa.fit_fractions_with_propagated_errors([
    (components[1], components[0]),
    (components[2], components[0]),
    (components[3], components[0]),
    (components[4], components[0]),
], phsp_set, fit_result)

In [ ]:
for fraction in fit_fractions:
    print(fraction.name.replace(';', '\n  '))
    print(fraction.value, '+/-', fraction.error)
    print()

And, please, feel free to
[provide feedback](https://github.com/ComPWA/pycompwa/issues/new)
or [contribute](https://compwa.github.io/contribute.html) ;)